# Modelling irregular bodies shape via ANNs
In this notebook we explore the possibility to use ANNs to represent the generic shape and density of an irregular body and be trained to reproduce a known gravitational potential field.

To get statically stable asteroids we use results from MPIA work by Francesco Biscani obtained during simulation of protoplanetary formation made by large n-body simulations. Data are included as a submodule in the git project.

To run this notebook create a conda environment using the following commands:
```
 conda create -n geodesyann python=3.8 ipython scikit-learn numpy h5py matplotlib
 conda install -c open3d-admin open3d
```

And you will need pytorch (CPU is enough) for the ANN part


In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from torch import nn
import torch
# For debugging and development purposes this is now set to float64 ... change for speed on GPUs
torch.set_default_tensor_type(torch.DoubleTensor)

%matplotlib notebook

# Loading and visualizing an asteroid as a point cloud

In [ ]:
# We import the data from MPIA containing pseudo-stable asteroid shapes
f = h5py.File('sample_vis_data/sample_01/state_10567.hdf5','r')
f2 = h5py.File('sample_vis_data/sample_01/global.hdf5', 'r')

In [ ]:
# The file state_ ... contains the positions of all particles as well as the indices
# of those belonging to a cluster. Here we extract the largest ones.
dims = [(len(f[cluster][()]), cluster) for cluster in f.keys() if 'cluster' in cluster]
largest_clusters = sorted(dims,reverse=True)

In [ ]:
# We have ordered the largest asteroids, we now extract positions for one in particular
rank = 4
print("Target: ", largest_clusters[rank][1])
# The particles idxs for this cluster
idx = f[largest_clusters[rank][1]][()]
# The particle radius
radius = f2['radius'][()]
# Particle positions
x_raw = f['x'][()][idx]
y_raw = f['y'][()][idx]
z_raw = f['z'][()][idx]
print("Diameter: ", 2 * radius)

In [ ]:
from sklearn.neighbors import NearestNeighbors
# We put xyz in a different shape (point_cloud)
point_cloud = np.append(x_raw, np.append(y_raw,z_raw))
point_cloud = point_cloud.reshape((3,len(x_raw)))
point_cloud = np.transpose(point_cloud)

nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(point_cloud)
distances, indices = nbrs.kneighbors(point_cloud)

print("Minimum distance between particles: ", min(distances[:,1]))
print("Maximum distance between particles: ", max(distances[:,1]))

# We take out particles that are not "touching" at least two neighbours
unstable_points = np.where(distances[:, 3]> 2 * radius * 1.01)[0]
print("Number of unstable points: ", len(unstable_points))
x = np.delete(x_raw, unstable_points, 0)
y = np.delete(y_raw, unstable_points, 0)
z = np.delete(z_raw, unstable_points, 0)

In [ ]:
# We subtract the mean so that the origin is the center of figure
x = x - np.mean(x)
y = y - np.mean(y)
z = z - np.mean(z)
# We normalize so that the axes are at most one
max_value = max([max(abs(it)) for it in [x,y,z]])
x = x / max_value
y = y / max_value
z = z / max_value
plot_radius = radius /  max_value  * 3000
# We put xyz in a different shape (point_cloud)
point_cloud = np.append(x, np.append(y,z))
point_cloud = point_cloud.reshape((3,len(x)))
point_cloud = np.transpose(point_cloud)

### Visualization via matplotlib

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection='3d')

# We color the points w.r.t. their distance to the center
color = [np.linalg.norm([point]) for point in point_cloud] 
# And visualize the masses
ax = plt.axes(projection='3d')
ax.scatter(x, y, z, color = 'k', s = plot_radius/2, alpha=0.5)
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])
ax.view_init(elev=45., azim=125.)

### Visualization via open3d

In [ ]:
import numpy as np
import open3d as o3d

colors = np.exp(-np.array(color))
colors = np.append(np.append(colors, colors), colors).reshape((3, len(color))).transpose()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)
pcd.colors = o3d.utility.Vector3dVector(colors)


hull, _ = pcd.compute_convex_hull()
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((1, 0, 0))
o3d.visualization.draw_geometries([pcd, hull_ls])

# Computing the gravitational potential of an asteroid (point cloud) in a point 
The Canvendish constant is not included (or $G=1$), so that we have:
$$
U_L = - \sum_{i=1}^N \frac{m_i}{|\mathbf x - \mathbf r_i|}
$$
where, assuming the asteroid with a unitary mass $m_i = 1/N$, hence:
$$
U_L = - \frac 1N \sum_{i=1}^N \frac{1}{|\mathbf x - \mathbf r_i|}
$$

In [ ]:
def U_L(r, point_cloud):
    retval=torch.empty(len(r),1)
    # Only for the points inside we accumulate the integrand (MC method)
    for i, radius in enumerate(r):
        retval[i] = torch.mean(1./torch.norm(torch.sub(point_cloud,radius), dim=1))
    return - retval 

In [ ]:
tcp = torch.tensor(point_cloud)
r = torch.rand(100,3)

In [ ]:
%timeit U_L(r, point_cloud=tcp)

# Representing the asteroid via a neural network


## Approach N.1: A FFNN represents the asteroid surface
In this approach we try to represent the asteroid minimalistically via its surface. Assuming an internal uniform density we will then be able to compute the potential generated at some point.

The FFNN will get three inputs (expressing in a continuous fashion a direction as a unit vector) and output the distance of the asteroid surface from the origin in that particular direction.

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.uniform_(m.weight.data, -1, 1)
        nn.init.uniform_(m.bias.data, -0.1, 0.1)

# Initializing a FFNN with three inputs (cartesian components of the unit direction vector) 
# and one output (the asteroid radius)
model = nn.Sequential(
          nn.Linear(3,10),
          nn.Tanh(),
          nn.Linear(10,10),
          nn.Tanh(),
          nn.Linear(10,10),
          nn.ReLU(),
          nn.Linear(10,1),
          nn.Sigmoid(),
        )

# Applying our weight initialization
_ = model.apply(weights_init)



In [ ]:
# Creating N equally spaced unit vectors on a sphere (to then plot r)
N = 40
inputs = []
u, v = torch.meshgrid((torch.linspace(0., 1., N), torch.linspace(0, 1, N)))
theta, phi = 2 * np.pi * u, torch.acos(2.*v-1.)
x = torch.sin(phi)*torch.cos(theta)
y = torch.sin(phi)*torch.sin(theta)
z = torch.cos(phi)
inputs = torch.cat((x.view(-1,1),y.view(-1,1), z.view(-1,1)), dim = 1)


In [ ]:
# For plotting no gradients are needed
r = model(inputs).detach()

In [ ]:
# We plot the visual representation of the asteroid
THETA, PHI = theta, phi
R = r.view((N,N))

X = R * torch.sin(PHI) * torch.cos(THETA)
Y = R * torch.sin(PHI) * torch.sin(THETA)
Z = R * torch.cos(PHI)
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection='3d')
plot = ax.plot_surface(
    X.numpy(), Y.numpy(), Z.numpy(), rstride=1, cstride=1, cmap=plt.get_cmap('jet'),
    linewidth=0, antialiased=False, alpha=0.5)
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])

## Computing the gravitational potential of the asteroid ANN model

We have a representation of the asteroid surface (and hence volume) given by our ANN, we may then assume a uniform internal density and  compute the resulting potential at a given point $\mathbf x$ as:

$$
U_P = \int_V \rho \frac1{|\mathbf r - \mathbf x|} dV = \rho \int_V \frac{dV}{|\mathbf r - \mathbf x|} 
$$

to compute the integral we use Monte Carlo integration methods, that is we sample N points in a volume V = [-1,1]x[-1,1]x[-1,1] that we know contains our asteroid (as ensured by the sigmoid activation function of the last layer). We then approximate:

$$
U_P(\mathbf x) = \rho\frac V N  \sum_i^N \left\{\begin{array}{ll}\frac{1}{|\mathbf r_i - \mathbf x|} & \mbox{inside the asteroid} \\ 0 &  \mbox{otherwise}  \end{array}\right.
$$

Since $\rho = \frac MV$ is itself an integral being $V = \int_V dV \approx \frac N{N_{in}V}$ we get:

$$
U_P(\mathbf x) = \frac 1{N_{in}}  \sum_i^N \left\{\begin{array}{ll}\frac{1}{|\mathbf r_i - \mathbf x|} & \mbox{inside the asteroid} \\ 0 &  \mbox{otherwise}  \end{array}\right.
$$

In [ ]:
def U_P(r, model, N = 300):
    # We generate randomly 300 points in the [-1,1]^3 bounds
    points = torch.rand(N,3) * 2 - 1
    # We compute their norms
    norms = torch.norm(points, dim=1)
    # We compute if they are inside or outside the asteroid represented by the FFNN model
    inside = norms.view(-1,1) < model(points/norms.view(-1,1))
    # Only for the points inside we accumulate the integrand (MC method)
    retval = torch.sum(1./torch.norm(r - points[inside.view(-1,)], dim=1))
    return  - retval / torch.sum(inside)

In [ ]:
# Helper class to represent a sphere model
class sphere:
    def __init__(self, N = 300):
        self.N = N
    def __call__(self, x):
        return torch.ones(self.N,1)

spherical_asteroid = sphere(N = 3000)
NN_asteroid = model

target_r = torch.tensor([[1.,1.,1.]])

# We print the values of the potential in r of a sphere, of the ground truth asteroid (point cloud)
# and of the asteroid as represented by the NN
print("Potential of the asteroid represented by the NN (MC): ", U_P(target_r, NN_asteroid, N=3000).item())
print("Potential of a spherical asteroid (MC): ", U_P(target_r, spherical_asteroid, N=3000).item())
print("Potential of a spherical asteroid (Real): ", (-1./torch.norm(target_r)).item())
print("Potential of the point cloud: (Real)", U_L(target_r, point_cloud=tcp)[0][0].item())

In [ ]:
# All good, but can we get the gradient of the above potential w.r.t. the weights? -> NOPE
# So the only way to evolve this network is EVOLUTION!!!
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://insight-quality.com/wp-content/uploads/2018/04/problem.jpg")

## Approach N.2: A FFNN represents the asteroid density
In this approach we try to represent the asteroid density $\rho$ directly! This may seem worse
but, in terms of gradients, leads to a more learnable problem.

In [ ]:
# Network architecture
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.uniform_(m.weight.data, -1, 1)
        nn.init.uniform_(m.bias.data, -0.1, 0.1)

# Initializing a FFNN with four inputs and one output (the density).
# The four inputs are ix, iy, iz, r corresponding to the cartesian point P = r [ix, iy, iz]
model = nn.Sequential(
          nn.Linear(4,30),
          nn.ReLU(),
          nn.Linear(30,30),
          nn.ReLU(),
          nn.Linear(30,30),
          nn.ReLU(),
          nn.Linear(30,30),
          nn.ReLU(),
          nn.Linear(30,10),
          nn.ReLU(),
          nn.Linear(10,1),
          nn.Sigmoid(),
        )

# Applying our weight initialization
_  = model.apply(weights_init)

In [ ]:
# Converts a cartesian position x,y,z into a network input ix, iy, iz, r
def cart2inputs(x):
    unit_x = x / torch.norm(x,dim=1).view(-1,1)
    return torch.cat((unit_x, torch.norm(x,dim=1).view(-1,1)), dim=1)

In [ ]:
# To visualize the network output in the box [-1,1]^3 we compute the NN output on a grid 
x = torch.linspace(-1,1,20)
y = torch.linspace(-1,1,20)
z = torch.linspace(-1,1,20)
X, Y, Z = torch.meshgrid((x,y,z))
nn_inputs = torch.cat((X.reshape(-1,1), Y.reshape(-1,1), Z.reshape(-1,1)), dim=1)
nn_inputs = cart2inputs(nn_inputs)
RHO = model(nn_inputs).detach()


In [ ]:
# Here we plot it
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X.reshape(-1,1), Y.reshape(-1,1), Z.reshape(-1,1), marker='.', c=RHO, s=100, alpha=0.2)
plt.title("Believe me I am an asteoroid!")

## Computing the gravitational potential of the asteroid ANN model
Having the mass density $\rho$ we may now compute the potential field via a monte carlo integration approach.
In the naive implementation, we will sample at random $N$ points to compute:
$$
U_P = - \int_V \frac\rho r dV 
$$
The volume V is the cube $[-1,1]^3$, with a volume of 8. We thus approximate the above integral as:
$$
U_P(\mathbf r) \approx \frac 8N \sum_i \frac {\rho_i}{|\mathbf r-\mathbf x_i|} 
$$
where $\mathbf x_i$ are $N$ uniformly randomly sampled within the cube -> PLEASE IMPROVE ME!

In [ ]:
# This computes the potential at the points r (e.g. torch.tensor([[np.sqrt(2),np.sqrt(2),0], [10,0,0]]))
# from the model
def U_P2(r, model, N = 3000):
    # We generate randomly points in the [-1,1]^3 bounds
    mc_points = torch.rand(N,3) * 2 - 1
    nn_inputs = cart2inputs(mc_points)
    rho = model(nn_inputs)
    retval=torch.empty(len(r),1)
    # Only for the points inside we accumulate the integrand (MC method)
    for i, radius in enumerate(r):
        retval[i] = torch.sum(rho/torch.norm(radius - mc_points, dim=1).view(-1,1)) / N
    return  - 8 * retval

In [ ]:
def plot_chain(chain,values):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    x,y,z,c = [],[],[],[]
    for i,t in enumerate(chain):
        t = t.squeeze()
        x.append(t[0].item())
        y.append(t[1].item())
        z.append(t[2].item())
        c.append(values[i].item())
    ax.scatter3D(x,y,z,c=c,s=1)
    ax.set_xlim((-1.25,1.25))
    ax.set_ylim((-1.25,1.25))
    ax.set_zlim((-1.25,1.25))
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

def check_boundaries(points,values):
    r1 = torch.add(points[:,0] > 1.0,points[:,0] < -1.0)
    r2 = torch.add(points[:,1] > 1.0,points[:,1] < -1.0)
    r3 = torch.add(points[:,2] > 1.0,points[:,2] < -1.0)
    r = r1 + r2
    r += r3
    values[r] = 0.0
#     for i,point in enumerate(points):
#         if torch.any(point > 1.0) or torch.any(point < -1.0):
#             values[i] = 0.0
    return values
        
    
def uniform_prior(N = 1):
    return torch.rand(N,3) * 2 - 1

def sampled_prior(rho_model,stepsize = 0.05,N_samples = 10000):
    #sample rho
    sample_points = uniform_prior(N_samples) #pick sample points
    nn_inputs = cart2inputs(sample_points) #compute transform 
    rho_values = rho_model(nn_inputs).squeeze() #compute rhos at points
    rho_values = rho_values / torch.norm(rho_values) #normalize values
    
    #find mean and var
    mean = torch.mv(sample_points.t(),rho_values) / N_samples #compute arithmetic mean
    print(mean)
    variance = torch.sum(((mean - sample_points) * (mean - sample_points)).t()*rho_values,dim=1) 
    
    #define gaussian with that
    dist = torch.distributions.MultivariateNormal(mean,stepsize * torch.diag(variance))
#     dist = torch.distributions.Uniform(torch.ones(3) * - 1,torch.ones(3))
    return dist

def U_P3(target_points, model, N = 3000,plot=False,verbose=False, stepsize = 0.05):
    prior = sampled_prior(model,stepsize = stepsize)
    sample_points = prior.sample([N])
    p = torch.exp(prior.log_prob(sample_points))
#     p = torch.ones(N) * 0.5 # TODO REMOVE THIS IF NOT USING UNIFORM PRIOR
    retval = torch.zeros(len(target_points),1)
    nn_inputs = cart2inputs(sample_points)
    rho = model(nn_inputs)
    
    if verbose:
        print("Sample points \n",sample_points)
        print("p \n",p)
        print("rho \n",rho)
    
    for i, target_point in enumerate(target_points):
        
        f_values = rho/torch.norm(target_point - sample_points, dim=1).view(-1,1)
        f_values = check_boundaries(sample_points,f_values)
        retval[i] = torch.sum(torch.div(f_values.squeeze(),p.squeeze())) / N
        
        if verbose:
            divs = torch.div(f_values,p).detach().numpy()
            fs = f_values.detach().numpy()
            pv = p.detach().numpy()
            print("Ps \t\t Fs \t\t Fs/Ps")
            for p_val,f_val in zip(p,f_values):
                print(f"{p_val.item():.8f},\t{f_val.item():.8f},\t{f_val.item()/p_val.item():.8f}")
            print("Sum of all fs",torch.sum(f_values).item())
            print("Sum of all ps",torch.sum(p).item())
            print("Divisors in torch:",torch.div(f_values.squeeze(),p.squeeze()))
            print("Sum of fs/ps",torch.sum(torch.div(f_values.squeeze(),p.squeeze())).item())
            print("N",N)
            print("Integral = ",retval[i].item())
        if plot:
            plot_chain(sample_points,f_values)
    return  -retval

In [ ]:
estimate = U_P3(torch.tensor([[2,2,2]]), model, N=10000, plot=True,verbose=False,stepsize=0.05).item()
print(estimate)

In [ ]:
import time
def show_result(method,result,baseline,N,start,end):
    t = end - start
    err = torch.mean(torch.abs(result-baseline))
    if len(result) > 1:
        print(f"Potential ({method:11s}):  err={err:.8f},{N:10d} steps,   t={t:4f}")
    else:
        print(f"Potential ({method:11s}): {result.item():.8f},   err={err.item():.8f},{N:10d} steps,   t={t:4f}")

points = 2*uniform_prior(256)
    
N = 1000000
start = time.time()
baseline = U_P2(points, model, N=N)
end = time.time()
show_result("MC Baseline",baseline,baseline,N,start,end)
print("\n--------------------------------------------------------\n")

In [ ]:
# for N in range(1000,52000,5000):
for N in range(100,10000,1000):
    start = time.time()
    vanilla_mc = U_P2(points, model, N=N)
    end = time.time()
    show_result("Vanilla MC",vanilla_mc,baseline,N,start,end)
    
print("\n--------------------------------------------------------\n")

# for N in range(1000,52000,5000):
for N in range(100,10000,1000):
    start = time.time()
    estimate = U_P3(points, model, N=N, stepsize=0.05)
    end = time.time()
    show_result("MC Gaussian",estimate,baseline,N,start,end)

In [ ]:
# The dataset is here produced. It consists on the values of the potentials in 100 points radnomly
# sampled around the asteroid and excluding those that are potentially inside.
targets = (torch.rand(100,3)*2-1)*1.1
targets[torch.norm(targets, dim=1) > 1]
labels = U_L(targets, tcp)

# Here we set some details of the training
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Here we perform steps of gradient descent
Let it run up to when its < 1e-3 to actually see something that resembles the original asteroid. When stuck increase the number of monte carlo samples or play around the learning rate.

In [ ]:
for i in range(5000):
    # Compute the loss
    predicted = U_P2(targets, model, N=30000)
    loss = loss_fn(predicted, labels)
    print(i, loss.item())
    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

In [ ]:
x = torch.linspace(-1,1,30)
y = torch.linspace(-1,1,30)
z = torch.linspace(-1,1,30)
X, Y, Z = torch.meshgrid((x,y,z))
nn_inputs = torch.cat((X.reshape(-1,1), Y.reshape(-1,1), Z.reshape(-1,1)), dim=1)
nn_inputs = cart2inputs(nn_inputs)
RHO = model(nn_inputs).detach()

colors = torch.cat((1-RHO, 1-RHO, 1-RHO, RHO), dim=1)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X.reshape(-1,1), Y.reshape(-1,1), Z.reshape(-1,1), marker='.', c=colors, s=150)
#ax.scatter(targets[:,0], targets[:,1], targets[:,2], color='y')
plt.title("Believe me I am an asteoroid!")

ax.set_xlim([-1,1])
ax.set_ylim([-1,1])
ax.set_zlim([-1,1])
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
#ax.view_init(elev=45., azim=125.)

In [ ]:
fig = plt.figure()
plt.plot((1-RHO).numpy(), '.')

# TODO list:

* Code efficiency -> move to GPU and make training scalable to more sample points / mc points.
* MC integration -> a better algorithm must exist to sample the points (low-discrepancy maybe?)
* Network architecture -> this is only a possible one. More depth? Different inputs? Use the NERF decomposition in harmonics?
* How to visualize and interpret the results? The matplotlib trick used here should be improved? How do we know how well the asteroid shape and internal mass distributionis learned?
* Propagate trajectories around the asteroid.
* Can we incorporate observations from real spacecraft trajectories in the loss? Or other data-fusion things?
* Training with gravity rather than potential?
* What happens for non uniform bodies? Can we prove that we are able to follow the inhomogenuities?